# import

In [14]:
import os
import gc
import random
import math
import time
import datetime
import shutil

import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn.metrics import log_loss

import category_encoders as ce
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn.functional as F

import warnings
warnings.filterwarnings("ignore")
import yaml
yaml.warnings({'YAMLLoadWarning': False})

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import common as com

# Setting

In [15]:
############################################################################
# set seed = 42
############################################################################
com.seed_everything(42)
############################################################################
# load config and set logger
############################################################################
with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

log_folder = config['IO_OPTION']['OUTPUT_ROOT'] + '/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_baseline_MLP.ipynb')

In [16]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
os.makedirs(MODEL_DIR, exist_ok=True)

# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/working/kaggle/Mechanisms-of-Action-Prediction/working/experiment/MLP/baseline/out_dir/config.yaml'

# dataload

In [17]:
train_features = pd.read_csv(INPUT_ROOT+'/train_features.csv')
train_targets_scored = pd.read_csv(INPUT_ROOT+'/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv(INPUT_ROOT+'/train_targets_nonscored.csv')
test_features = pd.read_csv(INPUT_ROOT+'/test_features.csv')
submission = pd.read_csv(INPUT_ROOT+'/sample_submission.csv')